## Collaborative Filtering
Collaborative filtering is a machine learning technique that predicts ratings awarded to items by users.

### Import the ALS class
In this exercise, you will use the Alternating Least Squares collaborative filtering algorithm to creater a recommender.

In [1]:
import os
os.environ["HADOOP_USER_NAME"] = "spark"
os.environ["SPARK_MAJOR_VERSION"] = "2"
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

### Load Source Data
The source data for the recommender is in two files - one containing numeric IDs for movies and users, along with user ratings; and the other containing details of the movies.

In [3]:
spark = SparkSession.builder.appName('python-recommenders').getOrCreate()
spark.conf.set('spark.executor.memory', '3g')
spark.conf.set('spark.executor.cores', '3')
spark.conf.set('spark.cores.max', '3')
spark.conf.set('spark.driver.memory','3g')
ratings = spark.read.csv('/user/maria_dev/data/ratings.csv', inferSchema=True, header=True)
movies = spark.read.csv('/user/maria_dev/data/movies.csv', inferSchema=True, header=True)
ratings.join(movies, "movieId").show()

+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|rating| timestamp|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|     31|     1|   2.5|1260759144|Dangerous Minds (...|               Drama|
|   1029|     1|   3.0|1260759179|        Dumbo (1941)|Animation|Childre...|
|   1061|     1|   3.0|1260759182|     Sleepers (1996)|            Thriller|
|   1129|     1|   2.0|1260759185|Escape from New Y...|Action|Adventure|...|
|   1172|     1|   4.0|1260759205|Cinema Paradiso (...|               Drama|
|   1263|     1|   2.0|1260759151|Deer Hunter, The ...|           Drama|War|
|   1287|     1|   2.0|1260759187|      Ben-Hur (1959)|Action|Adventure|...|
|   1293|     1|   2.0|1260759148|       Gandhi (1982)|               Drama|
|   1339|     1|   3.5|1260759125|Dracula (Bram Sto...|Fantasy|Horror|Ro...|
|   1343|     1|   2.0|1260759131|    Cape Fear (1991)|            Thriller|

### Prepare the Data
To prepare the data, split it into a training set and a test set.

In [4]:
data = ratings.select("userId", "movieId", "rating")
splits = data.randomSplit([0.7, 0.3])
train = splits[0].withColumnRenamed("rating", "label")
test = splits[1].withColumnRenamed("rating", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print "Training Rows:", train_rows, " Testing Rows:", test_rows

Training Rows: 69867  Testing Rows: 30137


### Build the Recommender
The ALS class is an estimator, so you can use its **fit** method to traing a model, or you can include it in a pipeline. Rather than specifying a feature vector and as label, the ALS algorithm requries a numeric user ID, item ID, and rating.

In [5]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="label")
model = als.fit(train)

### Test the Recommender
Now that you've trained the recommender, you can see how accurately it predicts known ratings in the test set.

In [6]:
prediction = model.transform(test)
prediction.join(movies, "movieId").select("userId", "title", "prediction", "trueLabel").show(100, truncate=False)

+------+--------------------------------+----------+---------+
|userId|title                           |prediction|trueLabel|
+------+--------------------------------+----------+---------+
|575   |Awfully Big Adventure, An (1995)|NaN       |4.0      |
|548   |Hudsucker Proxy, The (1994)     |3.1679552 |4.0      |
|588   |Hudsucker Proxy, The (1994)     |1.6704364 |3.0      |
|126   |Hudsucker Proxy, The (1994)     |3.3189821 |5.0      |
|299   |Hudsucker Proxy, The (1994)     |4.6013494 |4.5      |
|309   |Hudsucker Proxy, The (1994)     |4.1317673 |4.0      |
|358   |Hudsucker Proxy, The (1994)     |4.0440507 |5.0      |
|537   |Hudsucker Proxy, The (1994)     |4.0770736 |5.0      |
|380   |Hudsucker Proxy, The (1994)     |3.4695451 |4.0      |
|285   |Hudsucker Proxy, The (1994)     |2.8520844 |5.0      |
|86    |Hudsucker Proxy, The (1994)     |4.351843  |4.0      |
|19    |Hudsucker Proxy, The (1994)     |4.078237  |3.0      |
|514   |Hudsucker Proxy, The (1994)     |4.321982  |4.0

The data used in this exercise describes 5-star rating activity from [MovieLens](http://movielens.org), a movie recommendation service. It was created by GroupLens, a research group in the Department of Computer Science and Engineering at the University of Minnesota, and is used here with permission.

This dataset and other GroupLens data sets are publicly available for download at <http://grouplens.org/datasets/>.

For more information, see F. Maxwell Harper and Joseph A. Konstan. 2015. [The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015)](http://dx.doi.org/10.1145/2827872)